In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import time

In [ ]:
import os
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

In [ ]:
g_model = Sequential()
g_model.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
g_model.add(BatchNormalization())
g_model.add(Reshape((7,7,128)))
g_model.add(UpSampling2D())
g_model.add(Conv2D(64, kernel_size=5, padding='same'))
g_model.add(BatchNormalization())
g_model.add(Activation(LeakyReLU(0.02)))
g_model.add(UpSampling2D())

g_model.add(Conv2D(32, kernel_size=5, padding='same'))
g_model.add(BatchNormalization())
g_model.add(Activation(LeakyReLU(0.02)))

g_model.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
g_model.summary()

In [ ]:
d_model = Sequential()
d_model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28, 28, 1), padding='same'))
d_model.add(Activation(LeakyReLU(0.2)))
d_model.add(Dropout(0.3))
d_model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
d_model.add(Activation(LeakyReLU(0.2)))
d_model.add(Dropout(0.3))

d_model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
d_model.add(Activation(LeakyReLU(0.2)))
d_model.add(Dropout(0.3))

d_model.add(Flatten())
d_model.add(Dense(1, activation='sigmoid'))
d_model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])
d_model.trainable = False
d_model.summary()

In [ ]:
g_input = Input(shape=(100,))
dis_output = d_model(g_model(g_input))
gan = Model(g_input, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gan.summary()

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

data_ = X_train.shape[0]
rows = X_train.shape[1]
cloums = X_train.shape[2]

epoch= 1001
batch_size = 32
saving_interval = 200
history= []

X_train = X_train.reshape(data_, rows, cloums, 1).astype('float32')
#X_train = (X_train - 127.5) / 127.5
X_train = X_train / 255.0
true = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [ ]:
for i in range(epoch):
    start_time=time.time()
    idx = np.random.randint(0, X_train.shape[0], batch_size) # 0~ 60000 범위내에서 32개의 난수 생성
    imgs = X_train[idx] # X_train의 idx 32개를 저장
    d_loss_real = d_model.train_on_batch(imgs, true) # X_train의 32개는 모두 real 학습

    noise = np.random.normal(0, 1, (batch_size, 100)) # 0 ~ 1 범위내에서 (32, 100) 개의 난수 생성
    gen_imgs = g_model.predict(noise) # 생성자에 
    d_loss_fake = d_model.train_on_batch(gen_imgs, fake) 

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    g_loss = gan.train_on_batch(noise, true) 

    # 기록
    record = (epoch, d_loss[0], d_loss[1], g_loss[0], g_loss[1])
    history.append(record)

    #print('epoch:%d' %i, ' d_Loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss) 
    
    log_mesg = "%d elapsed : %f " % (i, (time.time() - start_time))
    log_mesg = "%s: [D loss: %f, acc: %f]" % (log_mesg, d_loss[0], d_loss[1])
    log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, g_loss[0], g_loss[1])
    
    print(log_mesg)

    if i % saving_interval == 0:
        noise = np.random.normal(0, 1, (25, 100))
        gen_imgs = g_model.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(4, 10, figsize=(15, 5))
        count = 0

        print(y_train[idx[:10]])

        for j in range(axs.shape[1]) :
            
            axs[0, j].axis('off')
            axs[0, j].imshow(gen_imgs[j].reshape((rows, cloums)), cmap='gray') 
            axs[1, j].axis('off')
            axs[1, j].imshow(imgs[j].reshape((rows, cloums)), cmap='gray')    
            axs[2, j].axis('off')
            axs[2, j].set_ylim([0, 1])
            axs[2, j].set_xlim([0, 1]) 
            axs[2, j].scatter(gen_imgs[j], gen_imgs[j], color='b')
            axs[3, j].axis('off')
            axs[3, j].set_ylim([0, 1])
            axs[3, j].set_xlim([0, 1])
            axs[3, j].scatter(imgs[j], imgs[j], color='r')
            
        fig.savefig("gan_images/gan_mnist_%d.png" % i)
        plt.show()
        plt.close(fig)

In [ ]:
model_dir = './Gan_model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
g_model.save_weights('my_g_model.h5')

In [ ]:
from pandas import Series, DataFrame
import pandas as pd 

df = DataFrame(history, columns=['epoch', 'd_loss', 'd_acc', 'g_loss', 'g_acc'])
df.plot(y=['d_loss', 'g_loss'])
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
true = np.ones((10, 1))
fake = np.zeros((10, 1))

np.random.seed(0)
#test
idx = np.random.randint(0, 32, 10) # 0~ 60000 범위내에서 32개의 난수 생성

imgs = X_train[idx] # X_train의 idx 32개를 저장
d_loss_real = d_model.train_on_batch(imgs, true) # X_train의 32개는 모두 real 학습
print(y_train[idx])

noise = np.random.normal(0, 1, (10, 100)) # 0 ~ 1 범위내에서 (32, 100) 개의 난수 생성
gen_imgs = g_model.predict(noise) # 생성자에 
d_loss_fake = d_model.train_on_batch(gen_imgs, fake) 

d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
g_loss = gan.train_on_batch(noise, true) 

# 기록
record = (epoch, d_loss[0], d_loss[1], g_loss[0], g_loss[1])
history.append(record)

log_mesg = "%d: [D loss: %f, acc: %f]" % (1, d_loss[0], d_loss[1])
log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, g_loss[0], g_loss[1])

print(log_mesg)


noise = np.random.normal(0, 1, (25, 100))
gen_imgs = g_model.predict(noise)
gen_imgs = 0.5 * gen_imgs + 0.5

fig, axs = plt.subplots(4, 10, figsize=(15, 5))

count = 0

for j in range(axs.shape[1]) :
    axs[0, j].axis('off')
    axs[0, j].imshow(gen_imgs[j].reshape((rows, cloums)), cmap='gray') 
    axs[1, j].axis('off')
    axs[1, j].imshow(imgs[j].reshape((rows, cloums)), cmap='gray')    
    axs[2, j].axis('off')
    axs[2, j].set_ylim([0, 1])
    axs[2, j].set_xlim([0, 1]) 
    axs[2, j].scatter(gen_imgs[j], gen_imgs[j], color='b')
    axs[3, j].axis('off')
    axs[3, j].set_ylim([0, 1])
    axs[3, j].set_xlim([0, 1])
    axs[3, j].scatter(imgs[j], imgs[j], color='r')
    
plt.show()
plt.close(fig)

In [ ]:
# def gan_train(epoch, batch_size, saving_interval) :
#     (X_train, _), (_,_) = mnist.load_data()
#     X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
#     X_train = (X_train -127.5) / 127.5
#     true = np.ones((batch_size, 1))
#     fake = np.zeros((batch_size, 1))
    
#     for i in range(epoch) :
#         idx = np.random.randint(0, X_train.shape[0], batch_size)
#         imgs = X_train[idx]
#         d_loss_real = d_model.train_on_batch(imgs, true)
        
#         noise = np.random.normal(0, 1, (batch_size, 100))
#         gen_imgs = g_model.predict(noise)
#         d_loss_fake = d_model.train_on_batch(gen_imgs, fake) 
        
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
#         g_loss = gan.train_on_batch(noise, true) 
        
#         print('epoch:%d' %i, ' dLoss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss) 
        
#         if i % saving_interval == 0:
#             #r, c = 5, 5
#             noise = np.random.normal(0, 1, (25, 100))
#             gen_imgs = g_model.predict(noise)

#             # Rescale images 0 - 1
#             gen_imgs = 0.5 * gen_imgs + 0.5

#             fig, axs = plt.subplots(5, 5)
#             count = 0
#             for j in range(5):
#               for k in range(5):
#                   axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
#                   axs[j, k].axis('off')
#                   count += 1
#             fig.savefig("gan_images/gan_mnist_%d.png" % i)